### NLP Setup

In [ ]:
import pickle
import sys
import spacy

# Load NLP
#sys.path.append('../')

with open('./nlp/nlp.pickle', 'rb') as f:
    nlp = pickle.load(f)

### Import the Template

In [ ]:
from app.templates.meat_sale.nl_template import nl_template

obs = nl_template['obligations']

print('OBLIGATIONS')
for ob in obs:
    print(f'{ob}: {obs[ob]}\n')

In [ ]:
from app.templates.meat_sale.symboleo.contract_template import get_template

contract_template = get_template()

sym_template = contract_template.to_sym()

sym_template_path = './app/templates/meat_sale/symboleo/symboleo_spec.txt'
with open(sym_template_path, 'w') as f:
    f.write(sym_template)

In [ ]:
from app.src.sentence_summarizer import SentenceSummarizer

summarizer = SentenceSummarizer(nlp)

### Reusable

In [ ]:
from app.templates.meat_sale.nl_template import parameters as meat_sale_parms

from app.src.rules.contract_spec.predicate_processor import PredicateProcessor
from app.src.rules.domain_model.domain_prop_processor import DomainPropProcessor

from app.src.rules.shared.configs import PredicateProcessorConfig
from app.src.rules.shared.configs import DomainPropProcessorConfig
# from app.src.rules.shared.predicate_extractor import PredicateExtractor

from app.src.norm_proposition_updater import NormPropositionUpdater
from app.classes.spec.predicate_function import PredicateFunctionHappens

from app.src.rules.contract_spec.timeframe.timeframe_extractor_builder import TimeFrameExtractorBuilder
from app.src.rules.domain_model.amount.amount_extractor_builder import AmountExtractorBuilder
from app.src.rules.domain_model.currency.currency_extractor_builder import CurrencyExtractorBuilder
from app.src.rules.domain_model.location.location_extractor_builder import LocationExtractorBuilder

norm_updater = NormPropositionUpdater()

### DELIVERY_TIMEFRAME

In [ ]:
delivered_event = contract_template.domain_model.events['delivered'].to_obj()
template = PredicateFunctionHappens(delivered_event)
dtf_extractor = TimeFrameExtractorBuilder.build(nlp, template)

dtf_pred_proc_config = meat_sale_parms['DELIVERY_TIMEFRAME'][0].config

dtf_processor = PredicateProcessor(dtf_pred_proc_config, dtf_extractor, norm_updater)

### DELIVERY_LOCATION

In [ ]:
dl_extractor = LocationExtractorBuilder.build(nlp)

dl_config = meat_sale_parms['DELIVERY_LOCATION'][0].config

dl_processor = DomainPropProcessor(dl_config, dl_extractor)

### PAYMENT_DETAILS

In [ ]:
pd_currency_extractor = CurrencyExtractorBuilder.build(nlp)

pd_curr_config = meat_sale_parms['PAYMENT_DETAILS'][1].config

pd_curr_processor = DomainPropProcessor(pd_curr_config, pd_currency_extractor)

In [ ]:
pd_amount_extractor = AmountExtractorBuilder.build(nlp)

pd_amt_config = meat_sale_parms['PAYMENT_DETAILS'][0].config

pd_amt_processor = DomainPropProcessor(pd_amt_config, pd_amount_extractor)

### PAYMENT_TIMEFRAME

In [ ]:
paid_event = contract_template.domain_model.events['paid'].to_obj()
paid_template = PredicateFunctionHappens(paid_event)
ptf_extractor = TimeFrameExtractorBuilder.build(nlp, paid_template)

ptf_config = meat_sale_parms['PAYMENT_TIMEFRAME'][0].config

ptf_processor = PredicateProcessor(ptf_config, ptf_extractor, norm_updater)

### INTEREST_DETAILS

In [ ]:
id_currency_extractor = CurrencyExtractorBuilder.build(nlp)

id_curr_config = meat_sale_parms['INTEREST_DETAILS'][1].config

id_curr_processor = DomainPropProcessor(id_curr_config, id_currency_extractor)

In [ ]:
id_amount_extractor = AmountExtractorBuilder.build(nlp)

id_amt_config = meat_sale_parms['INTEREST_DETAILS'][0].config

id_amt_processor = DomainPropProcessor(id_amt_config, id_amount_extractor)

### CONFIDENTIALITY_TIMEFRAME

In [ ]:
disclosed_event = contract_template.domain_model.events['disclosed'].to_obj()
disclosed_template = PredicateFunctionHappens(disclosed_event)
ctf_extractor = TimeFrameExtractorBuilder.build(nlp, disclosed_template)

ctf_config1 = meat_sale_parms['CONFIDENTIALITY_TIMEFRAME'][0].config

ctf_processor1 = PredicateProcessor(ctf_config1, ctf_extractor, norm_updater)

ctf_config2 = meat_sale_parms['CONFIDENTIALITY_TIMEFRAME'][1].config

ctf_processor2 = PredicateProcessor(ctf_config2, ctf_extractor, norm_updater)

### Contract Updater

In [ ]:
from app.src.contract_updater import ContractUpdater, ContractUpdateRequest
from app.src.processor_lookup import ProcessorLookup

processor_dict = {
    'DELIVERY_TIMEFRAME': [dtf_processor],
    'DELIVERY_LOCATION': [dl_processor],
    'PAYMENT_DETAILS': [pd_curr_processor, pd_amt_processor], 
    'PAYMENT_TIMEFRAME': [ptf_processor],
    'INTEREST_DETAILS': [id_amt_processor, id_curr_processor],
    'CONFIDENTIALITY_TIMEFRAME': [ctf_processor1, ctf_processor2],
}
processor_lookup = ProcessorLookup(processor_dict)

contract_updater = ContractUpdater(processor_lookup)

contract = contract_template

In [ ]:
customizations = [
    ('DELIVERY_TIMEFRAME', 'within 2 weeks'),
    ('DELIVERY_LOCATION', 'at the buyer\'s warehouse'),
    ('PAYMENT_DETAILS', '$100.00 CAD'),
    ('PAYMENT_TIMEFRAME', 'before April 17, 2022'),
    ('INTEREST_DETAILS', '10% of the amount owed'),
    ('CONFIDENTIALITY_TIMEFRAME', 'until 6 months after termination of the contract')
]

In [ ]:
for c in customizations:
    key = c[0]
    value = c[1]
    doc = nlp(value)

    req = ContractUpdateRequest(contract, key, value, doc)

    print(key)
    contract = contract_updater.update(req)

In [ ]:
print(contract.to_sym())